![Example Blending problem with 6 timestamps](blend-ex.png)

In [7]:
import numpy as np
import pandas as pd
from itertools import product

from pyomo.environ import *

num_timestamps = 6
alpha = 0.1  # fixed cost for using the pipelines
beta = 0.02

supply_tanks = ["s1", "s2"]
demand_tanks = ["d1", "d2"]
blending_tanks = ["b1", "b2", "b3", "b4", "b5", "b6", "b7", "b8"]
properties = ["q1"]
time_periods = list(range(num_timestamps))


supply_amnts = {'s1': [10, 10, 10, 0, 0, 0], 's2': [30, 30, 30, 0, 0, 0]}
demand_amnts = {'d1': [0, 0, 15, 15, 15, 15], 'd2': [0, 0, 15, 15, 15, 15]}

specs_suppl_init = {"s1":{"q1": 0.06}, "s2":{"q1": 0.26}}

specs_demand_up_bound = {"d1":{"q1": 0.16}, "d2":{"q1": 1}}
specs_demand_lower_bound = {"d1":{"q1": 0}, "d2":{"q1": 0}}

s_inv_lb = {'s1': 0, 's2': 0}  # inventory at supply and demand tanks are zero.
s_inv_ub = {'s1': 0, 's2': 0}
d_inv_lb = {'d1': 0, 'd2': 0}
d_inv_ub = {'d1': 0, 'd2': 0}

d_quals_lb = {'d1': 0, 'd2': 0}
d_quals_ub = {'d1': 0.16, 'd2': 0.1}
betaT_d = {'d1': 2, 'd2': 1}

# max inventory is 30 for the first 4 tanks and 20 for the second four tanks.
b_inv_ub = {"b1": 30, "b2": 30, "b3": 30, "b4": 30, "b5": 20, "b6": 20, "b7": 20, "b8": 20}

In [8]:
# Pipeline connections added manually
connections = {
    'supply_blend':{
        's1': ['b1', 'b2', 'b3', 'b4'],
        's2': ['b1', 'b2', 'b3', 'b4']
    },
    
    'supply_demand':{
        's1': [],
        's2': []
    },
    
    'blend_blend':{
        'b1': ['b5', 'b6', 'b7', 'b8'],
        'b2': ['b5', 'b6', 'b7', 'b8'],
        'b3': ['b5', 'b6', 'b7', 'b8'],
        'b4': ['b5', 'b6', 'b7', 'b8']
    },
    
    'blend_demand':{
        'b5': ['d1', 'd2'],
        'b6': ['d1', 'd2'],
        'b7': ['d1', 'd2'],
        'b8': ['d1', 'd2']
    }
}

In [13]:
# Model
model = ConcreteModel()

# Sets
model.supplies = Set(initialize=supply_tanks)
model.demands = Set(initialize=demand_tanks)
model.blenders = Set(initialize=blending_tanks)
model.properties = Set(initialize=properties)
model.time_periods = Set(initialize=time_periods)

In [16]:
# Parameters of the model
model.s_inv_lb = Param(model.supplies, initialize=s_inv_lb)
model.s_inv_ub = Param(model.supplies, initialize=s_inv_ub)
model.s_amounts = Param(model.supplies, initialize=supply_amnts)
model.d_quals_lb = Param(model.demands, initialize=d_quals_lb)
model.d_quals_ub = Param(model.demands, initialize=d_quals_ub)
model.d_inv_lb = Param(model.demands, initialize=d_inv_lb)
model.d_inv_ub = Param(model.demands, initialize=d_inv_ub)
model.d_amounts = Param(model.demands, initialize=demand_amnts)
model.betaT_d = Param(model.demands, initialize=betaT_d)
model.b_inv_ub = Param(model.blenders, initialize=b_inv_ub)

# Decision variables, Continuous
model.source_inv = Var(model.supplies, model.time_periods, domain=NonNegativeReals)
model.blend_inv = Var(model.blenders, model.time_periods, domain=NonNegativeReals)
model.demand_inv = Var(model.demands, model.time_periods, domain=NonNegativeReals)

# Concentration
model.prop_blend_inv = Var(model.properties, model.blenders, model.time_periods, domain=NonNegativeReals)

model.source_blend_flow = Var(model.supplies, model.blenders, model.time_periods, domain=NonNegativeReals)
model.blend_blend_flow = Var(model.blenders, model.blenders, model.time_periods, domain=NonNegativeReals)
model.blend_demand_flow = Var(model.blenders, model.demands, model.time_periods, domain=NonNegativeReals)

# Binary Variables
model.source_blend_bin = Var(model.supplies, model.blenders, model.time_periods, domain=Binary)
model.blend_blend_bin = Var(model.blenders, model.blenders, model.time_periods, domain=Binary)
model.blend_demand_bin = Var(model.blenders, model.demands, model.time_periods, domain=Binary)

'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and block.add_component().
'pyomo.core.base.param.IndexedParam'>) on block unknown with a new Component
(type=<class 'pyomo.core.base.param.IndexedParam'>). This is usually
indicative of a modelling error. To avoid this warning, use
block.del_component() and blo

In [12]:
model.s_amounts["s1"][0]
model.timestamps.prev(5)

4

In [19]:
def source_inv_rule(model, s, t):   # Rule 1
    if t == 0:
        return model.source_inv[s, t] == 0 # As we have empty inventory at t=0
    else:
        return model.source_inv[s, t] == model.source_inv[s, model.time_periods.prev(t)] \
                                        + model.s_amounts[s][model.time_periods.prev(t)] \
                                        - sum(model.source_blend_flow[s, j, model.time_periods.prev(t)] for j in model.blenders)

model.source_inv_rule = Constraint(model.supplies, model.time_periods, rule=source_inv_rule)

In [21]:
def blending_inv_rule(model, j, t):   # Rule 2
    if t == 0:
        return model.blend_inv[j, t] == 0 # As we have empty inventory at t=0
    else:
        return model.blend_inv[j, t] == model.blend_inv[j, model.time_periods.prev(t)] \
                                    + sum(model.source_blend_flow[s, j, model.time_periods.prev(t)] for s  in model.supplies) \
                                    + sum(model.blend_blend_flow[jp, j, model.time_periods.prev(t)] for jp in model.blenders) \
                                    - sum(model.blend_blend_flow[j, jp, model.time_periods.prev(t)] for jp in model.blenders) \
                                    - sum(model.blend_demand_flow[j, d, model.time_periods.prev(t)] for d  in model.demands)

model.blending_inv_rule = Constraint(model.blenders, model.time_periods, rule=blending_inv_rule)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [22]:
def demand_inv_rule(model, p, t):   # Rule 3
    if t == 0:
        return model.demand_inv[p, t] == 0 # As we have empty inventory at t=0
    else:
        return model.demand_inv[p, t] == model.demand_inv[p, model.time_periods.prev(t)] \
                                    + sum(model.blend_demand_flow[j, p, model.time_periods.prev(t)] for j in model.blenders) \
                                    - model.d_amounts[p][model.time_periods.prev(t)]

model.demand_inv_rule = Constraint(model.demands, model.time_periods, rule=demand_inv_rule)

In [23]:
M = 99999999   # Some very high number
def source_blend_flow_rule(model, s, j, t):  # Rule 4
    return model.source_blend_flow[s, j, t] <= M * model.source_blend_bin[s, j, t]

model.source_blend_flow_rule = Constraint(model.supplies, model.blenders, model.time_periods, rule=source_blend_flow_rule)

In [24]:
def binary_var_contraint(model, s, j, p, t):  # Rule 5
    return model.source_blend_bin[s, j, t] <= 1 - model.blend_demand_bin[j, p, t]

model.binary_var_contraint = Constraint(model.supplies, model.blenders, model.demands, model.time_periods, rule=binary_var_contraint)

In [26]:
def concentration_constraint(model, q, j, t):  # Rule 6
    if t == 0:
        return model.prop_blend_inv[q, j, t] == 0 # As inventory at t=0 is 0.
    else:
        return model.prop_blend_inv[q, j, t] * model.blend_inv[j, t] == model.prop_blend_inv[q, j, model.time_periods.prev(t)] * model.blend_inv[j, model.time_periods.prev(t)] \
                                                    + sum(specs_suppl_init[s][q] * model.source_blend_flow[s, j, model.time_periods.prev(t)] for s in model.supplies) \
                                                    + sum(model.prop_blend_inv[q, jp, model.time_periods.prev(t)] * model.blend_blend_flow[jp, j, model.time_periods.prev(t)] for jp in model.blenders) \
                                                    - sum(model.prop_blend_inv[q, j,  model.time_periods.prev(t)] * model.blend_blend_flow[j, jp, model.time_periods.prev(t)] for jp in model.blenders) \
                                                    - sum(model.prop_blend_inv[q, j,  model.time_periods.prev(t)] * model.blend_demand_flow[j, p, model.time_periods.prev(t)] for p in model.demands) \

model.concentration_constraint = Constraint(model.properties, model.blenders, model.time_periods, rule=concentration_constraint)

(type=<class 'pyomo.core.base.constraint.IndexedConstraint'>) on block unknown
with a new Component (type=<class
'pyomo.core.base.constraint.IndexedConstraint'>). This is usually indicative
of a modelling error. To avoid this warning, use block.del_component() and
block.add_component().


In [27]:
def specification_constraint_part1(model, q, p, j, t): # Left part of the Rule 7
    return specs_demand_lower_bound[p][q] - M * (1 - model.blend_demand_bin[j, p, t]) <= model.prop_blend_inv[q, j, t]

def specification_constraint_part2(model, q, p, j, t):   # Right part of the Rule 7
    return specs_demand_up_bound[p][q] + M * (1 - model.blend_demand_bin[j, p, t]) >= model.prop_blend_inv[q, j, t]

model.specification_constraint_part1 = Constraint(model.properties, model.demands, model.blenders, model.time_periods, rule=specification_constraint_part1)
model.specification_constraint_part2 = Constraint(model.properties, model.demands, model.blenders, model.time_periods, rule=specification_constraint_part2)

In [28]:
def objective_func(model):
    return  sum(
                sum(  
                      sum(alpha * model.source_blend_bin[s, j, t] + beta * model.source_blend_flow[s, j, t] for s in model.supplies) \
                    + sum(alpha * model.blend_blend_bin[j, jp, t] + beta * model.blend_blend_flow[j, jp, t] for jp in model.blenders) \
                    + sum(alpha * model.blend_demand_bin[j, p, t] + beta * model.blend_demand_flow[j, p, t] for p in model.demands) 
                for t in model.time_periods) 
            for j in model.blenders)

model.obj_fn = Objective(rule=objective_func, sense=minimize)

In [33]:
# Solve the model
solver = SolverFactory('gurobi')
solver.solve(model, tee=True)

Set parameter Username
Academic license - for non-commercial use only - expires 2025-03-27
Read LP format model from file C:\Users\manik\AppData\Local\Temp\tmpz_3rz9kv.pyomo.lp
Reading time = 0.01 seconds
x1: 560 rows, 1272 columns, 1980 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 12th Gen Intel(R) Core(TM) i7-1260P, instruction set [SSE2|AVX|AVX2]
Thread count: 12 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 560 rows, 1272 columns and 1980 nonzeros
Model fingerprint: 0x38cdb3f8
Model has 40 quadratic constraints
Variable types: 696 continuous, 576 integer (576 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+08]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [6e-02, 3e-01]
  Objective range  [2e-02, 1e-01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+08]
Presolve removed 428 rows and 664 columns
Presolve time: 0.00s
Presolved: 1508 rows, 937 columns,

{'Problem': [{'Name': 'x1', 'Lower bound': 3.699999999978177, 'Upper bound': 3.699999999978177, 'Number of objectives': 1, 'Number of constraints': 600, 'Number of variables': 1272, 'Number of binary variables': 576, 'Number of integer variables': 576, 'Number of continuous variables': 696, 'Number of nonzeros': 1980, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Return code': '0', 'Message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Termination condition': 'optimal', 'Termination message': 'Model was solved to optimality (subject to tolerances), and an optimal solution is available.', 'Wall time': '0.10899996757507324', 'Error rc': 0, 'Time': 0.439145565032959}], 'Solution': [OrderedDict({'number of solutions': 0, 'number of solutions displayed': 0})]}

In [36]:
flow_src_blend = [[[model.source_blend_flow[s, b, t].value for b in model.blenders] for s in model.supplies] for t in model.time_periods]
flow_blend_blend = [[[model.blend_blend_flow[b1, b2, t].value for b2 in model.blenders] for b1 in model.blenders] for t in model.time_periods]
flow_blend_demand = [[[model.blend_demand_flow[b, p, t].value for p in model.demands] for b in model.blenders] for t in model.time_periods]

src_blend_bin_var = [[[model.source_blend_bin[s, b, t].value for b in model.blenders] for s in model.supplies] for t in model.time_periods]
blend_blend_bin_var = [[[model.blend_blend_bin[b1, b2, t].value for b2 in model.blenders] for b1 in model.blenders] for t in model.time_periods]
blend_demand_bin_var = [[[model.blend_demand_bin[b, p, t].value for p in model.demands] for b in model.blenders] for t in model.time_periods]

supply_inv_t = [[model.source_inv[s, t].value for s in model.supplies] for t in model.time_periods]
blend_inv_t = [[model.blend_inv[b, t].value for b in model.blenders] for t in model.time_periods]
demand_inv_t = [[model.demand_inv[p, t].value for p in model.demands] for t in model.time_periods]

print("---------------- Flow Variables Start -----------------")
for t in range(num_timestamps):
    print(f"\n For Time period t{t}:")
    print("Source -- Blend")
    for s in range(len(model.supplies)):
        print(flow_src_blend[t][s])
    print("Blend -- Blend")
    for b in range(len(model.blenders)):
        print(flow_blend_blend[t][b])
    print("Blend -- Products")
    for b in range(len(model.blenders)):
        print(flow_blend_demand[t][b])

print("------------------ Flow Variables End ----------------------")

print("---------------- Binary Variables Start -----------------")
for t in range(num_timestamps):
    print(f"\n For time period t{t}:")
    print("Source -- Blend")
    for s in range(len(model.supplies)):
        print(src_blend_bin_var[t][s])
    print("Blend -- Blend")
    for b in range(len(model.blenders)):
        print(blend_blend_bin_var[t][b])
    print("Blend -- Products")
    for b in range(len(model.blenders)):
        print(blend_demand_bin_var[t][b])
print("------------------ Binary Variables End ----------------------")


print("---------------- Inventory Variables Start -----------------")
for t in range(num_timestamps):
    print(f"\n For time period t{t}:")
    print("Supply Inventory")
    print(supply_inv_t[t])
    print("Blending Inventory")
    print(blend_inv_t[t])
    print("Demand Inventory")
    print(demand_inv_t[t])
print("------------------ Inventory Variables End ----------------------")
    
print("Objective Function Value:", model.obj_fn())

---------------- Flow Variables Start -----------------

 For Time period t0:
Source -- Blend
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Blend -- Blend
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Blend -- Products
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]
[0.0, 0.0]

 For Time period t1:
Source -- Blend
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
Blend -- Blend
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,